In [34]:
import requests 
from bs4 import BeautifulSoup
import csv
import re

URL = "https://www.whatmobile.com.pk/"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 

products = soup.findAll("li", attrs = {'class': 'product'})
mobiles = []

for product in products:
    mobile = {}
    
    this_mobile = requests.get(URL+product.find('a',attrs = {'class': 'BiggerText'})['href'])
    this_mobile = BeautifulSoup(this_mobile.content, 'html5lib')
    
    mobile['Name'] = this_mobile.find('h1', attrs = {'class': 'hdng3'}).text
    
    items = this_mobile.select(".specs > tbody > tr")    
    for item in items:
        processor_cpu = item.find('th', text = re.compile('CPU'))
        processor_chipset = item.find('th', text = re.compile('Chipset'))
        ram = item.find('th', text = re.compile('Built-in'))
        display_size = item.find('th', text = re.compile('Size'))
        #ram = item.find('th', text = re.compile('Built-in'))
        if processor_cpu != None:
            processor_cpu = processor_cpu.parent.find(attrs = {'class': 'specs-value'}).text
            mobile['CPU'] = processor_cpu
        if processor_chipset != None:
            processor_chipset = processor_chipset.parent.find(attrs = {'class': 'specs-value'}).text.strip()
            mobile['Chipset'] = processor_chipset 
        if ram != None:
            ram = ram.parent.find(attrs = {'class': 'specs-value'}).text.strip()
            mobile['Built-in Memory'] = ram   
        if display_size != None:
            display_size = display_size.parent.find(attrs = {'class': 'specs-value'}).text.strip()
            mobile['Display Size'] = display_size 
            
    mobile['Price'] = product.find('span', attrs = {'class': 'PriceFont'}).text
    mobile['Image'] = URL+product.img['src']
    mobiles.append(mobile)

print("Mobile Detials are Saved in csv file. Open file to view it.")

filename = 'mobiledetails.csv'
with open(filename, 'w') as f:
    w = csv.DictWriter(f,['Name','CPU','Chipset','Built-in Memory','Display Size','Price','Image'], dialect="excel")
    w.writeheader()
    w.writerows(mobiles)

KeyboardInterrupt: 